Rules for the hackathon:

1. Need to execute all the tasks for full score.
2. Environment can be on your own choice(Google Colab, Jupiter, Data Bricks, etc).
3. Outputs are to be accurate.
4. Can be performed as a group.
5. Attedance will be considered.
6. Can use public resources(open book).

Github link to get file, we have file in different formats. Use one which you are comfortable in.

In [1]:
!git clone https://github.com/rohithpotluri/USDATA.git temp1

Cloning into 'temp1'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 12 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), done.


Task 0 (Given and TO DO): Print any 10 words from the bill document. 

Task 1 (TO DO) Find 10 most frequent words in the bill.

Task 2 (TO DO): Find 10 most frequent **wordle** words in the bill (note that **wordle** is a valid word that has five letters, such as grant, award, and funds. Btw, Wordle is a popular game :) https://www.nytimes.com/games/wordle/index.html

Task 3 (TO DO): Find all **wordles** that start and end with same letter.

Task 4 (TO DO): Find all **wordles** where all 5 letters are distinct.

--------------------------------------------------------------------------------


In [2]:
 # Install pyspark
!pip install pyspark
# Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Check Spark Session Information
spark
# Import a Spark function from library
from pyspark.sql.functions import col

import pandas as pd

# import all needed packages.
import pyspark
from pyspark import *

from pyspark.mllib.util import MLUtils
from pyspark.sql import SparkSession

import numpy as np

import pandas as pd
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 47 kB/s 
     |████████████████████████████████| 199 kB 66.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=90d6045068f816f1089194a33868753b2cedca09873cb647447639af75b901c5
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


Task 0: Find the number of list of 10 words in the doc. 

In [3]:
# data is moved to rdd "a" with lowering all the characters and removing special characters.
def lower_clean_str(x):
    char = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}—~“”’'
    lowercased_str = x.lower()
    for ch in char:
        lowercased_str = lowercased_str.replace(ch, '')
    return lowercased_str
# create Rdd (VAR1) from the file in Github
var1 = spark.sparkContext.textFile('/content/temp1/PLAW-117publ58 (1) (2).txt').map(lower_clean_str)
var1.top(10)



['‘‘§ 70205 national multimodal cooperative freight research',
 '‘‘§ 70204 multistate freight corridor planning',
 '‘‘§ 6703 national culvert removal replacement and restoration grant program',
 '‘‘§ 6702 local and regional project assistance',
 '‘‘§ 6701 national infrastructure project assistance',
 '‘‘§ 6504 incorporation of department of transportation',
 '‘‘§ 611 asset concessions and innovative finance assistance',
 '‘‘§ 5507 transportation workforce outreach program',
 '‘‘§ 5506 advanced transportation research initiative',
 '‘‘§ 5338 authorizations']

Task 1: Find 10 most frequent words in the bill.


In [4]:
# from pyspark.sql.functions import ltrim,rtrim,trim
# reg_exp="\\s+"
counts = var1.flatMap(lambda line: line.split(" ")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)\
             .sortBy(lambda x:x[1], False)
counts.take(10)

[('the', 27098),
 ('of', 18858),
 ('', 17383),
 ('and', 14169),
 ('to', 11192),
 ('a', 8701),
 ('in', 8312),
 ('for', 6462),
 ('under', 4978),
 ('or', 4743)]

Task 2: Find 10 most frequent wordle word in the bill (note that wordle is a valid word that has five letters, no digits, such as grant, award, and funds.

In [5]:
# wordle = counts.filter(lambda x:len(x[0])==5 & len(x[0])!=*(0-9))
# filter(lambda x:x[0]!="00058")
wordle = counts.filter(lambda x:len(x[0])==5)\
               .filter(lambda x: not x[0].isdigit())

sorted = wordle.sortBy(lambda x:x[1], False)
sorted.take(10)

[('under', 4978),
 ('shall', 4420),
 ('state', 1673),
 ('title', 1391),
 ('grant', 1300),
 ('after', 1196),
 ('which', 1182),
 ('other', 1098),
 ('funds', 935),
 ('carry', 723)]

Task 3: Find all words that start and end with same letter.

In [6]:
wordle.filter(lambda x:(x[0][0]==x[0][-1])).filter(lambda x: not x[0].isdigit()).take(10)

[('local', 447),
 ('level', 70),
 ('trust', 66),
 ('river', 39),
 ('legal', 29),
 ('sites', 24),
 ('dated', 15),
 ('rider', 14),
 ('steps', 13),
 ('seeks', 9)]

Task 4: Find all wordles where all 5 letters are distinct.

In [9]:
wordle.filter(lambda x:(x[0][0]!=x[0][1]!=x[0][2]!=x[0][3]!=x[0][4]))\
      .filter(lambda x:(x[0][1]!=x[0][0]!=x[0][2]!=x[0][3]!=x[0][4]))\
      .filter(lambda x:(x[0][2]!=x[0][0]!=x[0][1]!=x[0][3]!=x[0][4]))\
      .filter(lambda x:(x[0][3]!=x[0][0]!=x[0][1]!=x[0][2]!=x[0][4]))\
      .filter(lambda x:(x[0][4]!=x[0][0]!=x[0][1]!=x[0][2]!=x[0][3]))\
      .take(10)
# wordle.filter(lambda x:x[0].isdistinct()).take(10)

[('under', 4978),
 ('grant', 1300),
 ('after', 1196),
 ('which', 1182),
 ('other', 1098),
 ('funds', 935),
 ('years', 723),
 ('until', 707),
 ('later', 681),
 ('means', 663)]